In [6]:
%load_ext autoreload
%autoreload 2

# These directives will automatically reload any modules that are loaded from file.

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Assignment No.2
## CPSC 323

### Team Members:
* Maziar Bastani
* Cody Broxton
* Daniel Henderson

## Programming assignment

Write a program to read a postfix expression and compute its value. Variablse are strings of **one or more characters** and numbers are **one or more digits**. All tokens are separated with a **single space**.

### Sample I/O

```
Enter a postfix expression: ab 22 * c + $
     Enter the value of ab: 2
     Enter the value of c: 4
         Final value= 48
     
     Continue(y/n)? y
Enter a postfix expression: jerry 123 + tom * - $
     Enter the value of tom: 2
     Enter the value of jerry: 3
         Final value= -250
     
     Continue(y/n)? n
```

In [7]:
# To run this program in a regular terminal, export the entire notebook using the commands in the web interface:
# File > Download as > Python(.py)
# Move the file to your preferred location from the Downloads folder.
# Remove all the text above this cell, and the text in this cell, using your text editor.
# The first line of your 'fixed' program should be the "shebang" comment below this line. (#!/usr/bin/env python3)

In [8]:
#!/usr/bin/env python3
#

import operator

# declare globals
def setup():
    # tokens is a dict (dictionary, associative array)
    # that holds:
    #   variable names
    #   
    # and status about those tokens.
    global tokens
    tokens = {}
    tokens.clear()
    # token_counter just supplies an ordinal number
    # to variable names as they're declared in the RPN calculator
    # so the user interface can ask the user for their values 
    # in that same order.
    global token_counter
    token_counter = 1
    global user_string
    user_string = ''
    # user_string is the RPN equation the user enters as input
    global value_stack
    value_stack = []
    # value_stack is the RPN calculator's computation stack.
    #

    


For the Reverse Polish Notation, or RPN, calculator, a single string is entered by the user. This string should represent a valid RPN sequence of numbers and operations, terminated by a *$* (dollar sign). Numbers may be literal integers like *22* and *100*, or variable names that represent an integer. 

Operations required by the assignment are:
* + (addition)
* - (subtraction)
* * (multiplication)

Operations additionally implemented for our fun are:
* / (division)
* ^ (exponent)

These additional operators will require additional logic, since they will require floating point, rather than integer, numbers.

Each of these operators is binary, in that they take two operands. Addition, like **2 + 2** in _infix notation_, requires two operands. The operand on the right will be operated on (added to) using the operand on the left. A result of **4** will be the result of the addition operation. _Postfix notation_ does not change the need for two operands for each binary operator. In postfix notation and rpn, the operands come first, though, followed by the operation to perform: **2 2 +**. The result is, of course, the same. 

The benefit of postfix and rpn is that the order of operations in more complicated or chained equations requires no parentheses or grouping symbols. The infix equation **3 + 2 * 2** uses the rules of precedence to get the answer **7** or requires grouping symbols to perform the addition first, then the multiplication: **(3 + 2) * 2** to get the answer **10**. The postfix expression ** 3 2 + 2 * ** puts the 3 and 2 operands first, then the addition operation. This clearly indicates (after some experience with seeing it) that the addition of the first two numbers is performed first, then _that_ result is multiplied by 2. If multiplication of two and two is really desired first, the expression ** 2 2 * 3 + ** accomlishes this.

The result of this is that, just like ** 2 + ** isn't a valid expression with infix, neither is ** 2 + ** valid in postfix. While you can see, in an infix equation, whether every binary operator has two numbers or expressions, one on the left and one on the right, it takes a bit of practice to see when a postfix expression doesn't have enough operands. However, if an expression has two operands before an operator, and continues to have one more operand to the left than there are operators, the expression is valid.

Some valid expressions:
* 2 2 + 3 +    -- three operands, two operators
* 2 2 2 2 + + +   -- four operands, three operators
* 3 2 4 +         -- three operands, one operator. The 3 is never used. Only the 2 and 4 are added together.

But this is invalid:
* 2 + 2 2 +  -- the leftmost addition has only one operand to work with


The main run loop `run()` calls each step of the RPN parser. For debugging info, it can be called with the value `True`, which functions can check for and display more debugging info.

The `main()` function is the traditional entry point when a python program is run from the comand line. 

The two lines
```
if __name__ == '__main__':
    main()
```
are a test for a flag the python interpreter can use to see if the file invoked from the command line has a main function. If it does, that function is called. Otherwise, as is the case when a file is `import`ed, the file has a main function, but it's **not** the file the user has run, so the main function is not run.

This lets developers test a smaller project file and run its `main()` function with user input or other commands, but also import it as a module into larger projects as a collection of useful methods, without having the test user input run. 

As an example, this code may work very well as an RPN calculator, and we can test that by asking the user to enter an expression. Later, we may want a larger project to evaluate expressions using its user interface, meeting its specific needs. We **don't** need our RPN code to halt in the middle and ask the user to enter a string. We only want the **under-the-hood** objects and methods, not the basic text entry and reporting we have in our simpler test code. By not running `main()`, which calls the UI elements, we can use all the rest without the UI.

In [9]:
def tokenize(string):
    operands = 0
    if len(string) == 0:
        return 'done'
    for s in string:
        if s.isalpha():
            add_token(s)
            operands += 1
        elif s in ('+', '-', '*', '/'):
            # binary operators take two operands
            # see if there are enough operands
            if operands < 2:
                print("error out. not enough operands")
                return('error')
            else:
                # remove operand, since result will be pushed as new operand
                operands -= 1
        elif s in ('$'):
            return 'done'
        elif s.isspace():
            pass
        else:
            print('illegal character reached')
            return('error')
            


In [10]:
def add_token(character):
    global token_counter, tokens
    if character not in tokens:
        tokens[character] = [token_counter, 0]
        token_counter += 1


In [11]:
def user_input_pass():
    global show_debug
    global tokens,  user_string
    user_string = input('Enter a postscript expression with a $ at the end: ')
    if tokenize(user_string) == 'done':
        if show_debug:
            print(tokens)
            print('tokenized successfully')
    else:
        print('tokenizer error')
    # assign_values()


In [12]:
def assign_values():
    global show_debug
    global tokens
    sorted_tokens = sorted(tokens.items(), key=operator.itemgetter(1))
    for variable in sorted_tokens:
        keyval = variable[0]
        varval = input('Enter value of ' + keyval + ': ')
        # should error out if can't convert to a float
        tokens[keyval][1] = float(varval)
    if show_debug:
        print("Token dictionary:")
        print(tokens)
        print("Sorted tokens:")
        print(sorted_tokens)


In [13]:
def compute():
    global tokens, user_string, value_stack
    for char in user_string:
        if char.isalpha():
            value_stack.append(tokens[char][1])
        if char in ('+', '-', '*', '/'):
            # binary operators
            # check that stack can handle
            if len(value_stack) > 1:
                B = value_stack.pop()
                A = value_stack.pop()
                if char == '+':
                    value_stack.append(A + B)
                elif char == '-':
                    value_stack.append(A - B)
                elif char == '*':
                    value_stack.append(A * B)
                elif char == '/':
                    if B == 0:
                        print('error. Divide by zero.')
                        value_stack.append(float('NaN'))
                        return('error')
                    value_stack.append(A / B)
                elif char.isspace():
                    pass
                else:
                    print("Unimplemented binary operator. Calculations will be wrong")
                    value_stack.append(A)
            else:
                print('Error: Stack underflow.')
                return('error')


In [14]:
def show_result():
    global show_debug
    global value_stack, tokens, value_stack
    if show_debug:
        print(tokens)
        print(value_stack)
    print("\nFinal Value = " + str(value_stack.pop()))


In [ ]:
def run(debug=False):
    global show_debug
    show_debug = debug
    
    # Python equivalent of do-while loop
    while True:
        setup()    
        user_input_pass()
        assign_values()
        compute()
        show_result()
        if input("Continue(y/n)? ") != 'y':
            break

def main():
    if input("run in debug mode? ") != 'y':
        run()
    else:
        run(True)


if __name__ == '__main__':
    main()

In [20]:
run(True)

Enter a postscript expression with a $ at the end: ab+cd-*$
{'a': [1, 0], 'b': [2, 0], 'd': [4, 0], 'c': [3, 0]}
tokenized successfully
Enter value of a: 3
Enter value of b: 5
Enter value of c: 7
Enter value of d: 2
Token dictionary:
{'a': [1, 3.0], 'b': [2, 5.0], 'd': [4, 2.0], 'c': [3, 7.0]}
Sorted tokens:
[('a', [1, 3.0]), ('b', [2, 5.0]), ('c', [3, 7.0]), ('d', [4, 2.0])]
{'a': [1, 3.0], 'b': [2, 5.0], 'd': [4, 2.0], 'c': [3, 7.0]}
[40.0]

Final Value = 40.0
Continue(y/n)? n
